# LSH Running Testing by Double 2T
### Nguyen Minh Tuan
### Duong Minh Tung

Đảm bảo tất cả các thư viện cần dùng đã được import

In [1]:
## Import Necessary Libraries
import pandas as pd
import numpy as np
import time
from numba import cuda, jit
from datasketch import MinHash, MinHashLSH
import LSH_class

Tiến hành load data từ file CSV

In [2]:
## Load Data
# Load the CSV file
df = pd.read_csv('movies_with_genres.csv')
movies = {}
for index, row in df.iterrows():
    title = row['originalTitle']
    tags = str(row['genres']).split(', ')
    tags = [tag.strip() for tag in tags]
    movies[title] = tags

In [3]:
import random
from collections import defaultdict
from datasketch import MinHash

class MinHashLSH:
    def __init__(self, threshold=0.5, num_perm=128):
        self.threshold = threshold
        self.num_perm = num_perm
        self.bands = int(self.num_perm / (1.0 / self.threshold))
        self.rows = self.num_perm // self.bands
        self.buckets = [defaultdict(list) for _ in range(self.bands)]

    def _hash_band(self, minhash, band):
        start = band * self.rows
        end = start + self.rows
        return hash(tuple(minhash.hashvalues[start:end]))

    def insert(self, key, minhash):
        for i in range(self.bands):
            hash_value = self._hash_band(minhash, i)
            self.buckets[i][hash_value].append(key)

    def query(self, minhash):
        candidates = set()
        for i in range(self.bands):
            hash_value = self._hash_band(minhash, i)
            candidates.update(self.buckets[i][hash_value])
        return list(candidates)

Original LSH Implementation

In [4]:
## Original LSH Implementation
def original_lsh(movies, num_hashes=128):
    start = time.time()
    lsh = MinHashLSH(threshold=0.5, num_perm=num_hashes)

    for title, tags in movies.items():
        m = MinHash(num_perm=num_hashes)
        for tag in tags:
            m.update(tag.encode('utf8'))
        lsh.insert(title, m)  

    end = time.time()
    return lsh, end - start

In [10]:
# Example movies dataset
movies = {
    "movie1": {"action", "adventure", "comedy"},
    "movie2": {"drama", "romance", "comedy"},
    "movie3": {"action", "thriller", "crime"},
    "movie4": {"romance", "drama"},
    "movie5": {"action", "adventure", "thriller"}
}

# Create the LSH and insert the movies
lsh, duration = original_lsh(movies)

# Query for a similar movie
query_movie = {"action", 'drama', 'romance'}

# Create a MinHash object for the query movie
m = MinHash(num_perm=128)
for tag in query_movie:
    m.update(tag.encode('utf8'))

# Query the LSH
similar_movies = lsh.query(m)

print(f"Movies similar to the query: {similar_movies}")


Movies similar to the query: ['movie1', 'movie2', 'movie5', 'movie4', 'movie3']


Parallel LSH Implementation

In [3]:
## Parallel LSH Implementation
@jit
def custom_hash(x, seed):
    return (x * 0x9e3779b9 + seed) & 0xFFFFFFFF

@cuda.jit
def cuda_update(tags, minhashes, num_hashes):
    pos = cuda.grid(1)
    if pos < tags.shape[0]:
        tag = tags[pos]
        for i in range(num_hashes):
            tag_hash = custom_hash(tag, i)
            cuda.atomic.min(minhashes, (pos, i), tag_hash)

def update_minhash_gpu(tags, num_hashes):
    tags_gpu = cuda.to_device(np.array(tags, dtype=np.int32))
    num_tags = tags_gpu.size
    minhashes_gpu = cuda.to_device(np.full((num_tags, num_hashes), np.inf, dtype=np.float32))

    threads_per_block = 256
    blocks_per_grid = (num_tags + (threads_per_block - 1)) // threads_per_block

    cuda_update[blocks_per_grid, threads_per_block](tags_gpu, minhashes_gpu, num_hashes)

    return minhashes_gpu.copy_to_host()

def parallel_lsh(movies, num_hashes=128):
    start = time.time()
    lsh = LSH_class.LSH(num_hashes=num_hashes)

    unique_tags = set(tag for tags in movies.values() for tag in tags)
    tag_to_int = {tag: i for i, tag in enumerate(unique_tags)}

    int_tags_dict = {title: np.array([tag_to_int[tag] for tag in tags], dtype=np.int32) for title, tags in movies.items()}
    all_tags = np.concatenate(list(int_tags_dict.values()))
    tags_offset = np.cumsum([0] + [len(tags) for tags in int_tags_dict.values()])

    all_tags_gpu = cuda.to_device(all_tags)
    minhashes_gpu = cuda.to_device(np.full((all_tags.size, num_hashes), np.inf, dtype=np.float32))

    threads_per_block = 256
    blocks_per_grid = (all_tags.size + (threads_per_block - 1)) // threads_per_block

    cuda_update[blocks_per_grid, threads_per_block](all_tags_gpu, minhashes_gpu, num_hashes)

    all_minhashes = minhashes_gpu.copy_to_host()

    for i, (title, tags) in enumerate(movies.items()):
        start_idx = tags_offset[i]
        end_idx = tags_offset[i + 1]
        minhash_signature = np.min(all_minhashes[start_idx:end_idx], axis=0)
        lsh.insert(title, minhash_signature)

    end = time.time()
    return lsh, end - start

Điểm khác biệt giữa 2 phiên bản
1. Phương thức băm và cải tiến:

- Phiên bản gốc: Sử dụng thư viện datasketch với MinHash và MinHashLSH. Mỗi thẻ (tag) được mã hóa và cập nhật vào một đối tượng MinHash.
- Phiên bản song song: Sử dụng một hàm băm tùy chỉnh (custom_hash) và cập nhật giá trị băm nhỏ nhất cho mỗi thẻ (tag) bằng CUDA để tính toán song song.
- Phiên bản này không dùng datasketch mà tự triển khai hàm băm và cập nhật bằng GPU.

2. Xử lý song song:

- Phiên bản gốc: Xử lý tuần tự. Mỗi bộ thẻ (tag) được xử lý từng cái một.
- Phiên bản song song: Xử lý song song bằng CUDA. Các thẻ (tag) được chuyển đổi thành các số nguyên và các giá trị băm nhỏ nhất được cập nhật đồng thời trên GPU.

3. Hiệu suất:

- Phiên bản gốc: Thực hiện trên CPU, chậm với số lượng phim và thẻ (tag) lớn.
- Phiên bản song song: Sử dụng GPU để tăng tốc độ tính toán các giá trị băm, giúp cải thiện hiệu suất đáng kể khi xử lý một lượng lớn dữ liệu.

4. Cấu trúc dữ liệu:

- Phiên bản gốc: Mỗi phim có một đối tượng MinHash riêng.
- Phiên bản song song: Các thẻ (tag) được chuyển đổi thành số nguyên và tất cả các phim được xử lý cùng lúc bằng CUDA, sau đó các chữ ký MinHash được lấy từ giá trị băm nhỏ nhất.


In [4]:
## Comparison
num_hashes = 128

# Original LSH
# original_lsh_instance, original_time = original_lsh(movies, num_hashes)

# Parallel LSH
parallel_lsh_instance, parallel_time = parallel_lsh(movies, num_hashes)

# Print Results
# print(f"Original LSH Time: {original_time:.2f} seconds")
print(f"Parallel LSH Time: {parallel_time:.2f} seconds")

Parallel LSH Time: 124.59 seconds


Thời gian chạy:
+ Phiên bản gốc: ... s
+ Phiên bản đã được tối ưu GPU: 74s

In [5]:
# Parallel LSH
parallel_lsh_instance.show_signatures()

Key: Carmencita, Bands: [(1457684100.0,)]
Key: Chinese Opium Den, Bands: [(1457684100.0,)]
Key: Das boxende Känguruh, Bands: [(1457684100.0,)]
Key: Les forgerons, Bands: [(1457684100.0,)]
Key: Baignade en mer, Bands: [(1457684100.0,)]
Key: Barnet Horse Fair, Bands: [(1457684100.0,)]
Key: Bataille de neige, Bands: [(1457684100.0,)]
Key: Le bivouac, Bands: [(1457684100.0,)]
Key: Les blanchisseuses, Bands: [(1457684100.0,)]
Key: Les chevaux de bois, Bands: [(1457684100.0,)]


Signature ở đây sẽ có dạng Key, Band với 
+ Key là tên của bộ phim trong dataset
+ Band là số của tag sau khi được chia vào trong bucket

In [6]:
## Query Function
def query_lsh(lsh, query_str, tag_to_int, num_hashes):
    query = [tag.strip() for tag in query_str.split(',')]
    query_minhash = MinHash(num_perm=num_hashes)
    for tag in query:
        query_minhash.update(tag.encode('utf8'))
    result = lsh.query(query_minhash)
    return result

## Query Example
query_str = "Action, Adventure"
# original_result = query_lsh(original_lsh_instance, query_str, movies, num_hashes)
parallel_result = query_lsh(parallel_lsh_instance, query_str, movies, num_hashes)

# Print Query Results
# print("Original LSH Query Result:")
# print(original_result)

print("Parallel LSH Query Result:")
print(parallel_result)

TypeError: 'MinHash' object is not subscriptable